In [ ]:
import os 

In [1]:
%pwd

'c:\\Users\\pavan\\Desktop\\Fraud-Detection-E2E-ML-project-using-MLflow\\Notebooks'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path

In [4]:
from fraud_detection_project.constants import *
from fraud_detection_project.utils.common import *

In [8]:
class ConfigurationManager:
    # First setting up the environment by defining the file paths of configuration file, params file and schema file.  
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        # reading the parameters from the files defined
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        # Creating the root directory in the artifacts folder
        create_directories([self.config.artifacts_root])


    # Data Ingestion step
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            # We are accessing the file paths that are defined in the configuration yaml file
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
        )

        return data_ingestion_config

In [11]:
import os
import urllib.request as request
from fraud_detection_project import logger
from fraud_detection_project.utils.common import get_size

In [12]:
# So far, We have configured the structure of the data ingestion
# Now to execute the steps like downloading from the source url lets define the function to peform those actions

# initializing the data ingestion class
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # defining the function to download the csv file from the source url
    def download_file(self):
        # checks if the data file is already present or not, if not executes the function
        if not os.path.exists(self.config.local_data_file):
            # Extracting the information of the source URL like dataset file name and url headers
            filename, headers = request.urlretrieve(
                # we are retriving the url path and file name that is defined in the config file
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'